<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-deeplearning/blob/master/05_Prac_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5장. CNN

#1 CNN

https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/images/cnn.ipynb?hl=ko#scrollTo=jKgyC5K_4O0d

###1) MNIST 데이터셋 다운로드하고 준비하기

In [1]:
#!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) #데이터 건수, 이미지 높이, 이미지 너비, 컬러 채널
test_images = test_images.reshape((10000, 28, 28, 1)) #데이터 건수, 이미지 높이, 이미지 너비, 컬러 채널

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

11493376/11490434 [==============================] - 0s 0us/step


### 2) CNN 만들기

In [2]:
model = models.Sequential()
# 특징 추출 (Feature Extraction)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # 32 * 3 * 3 + 32 = 320
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 32 * 64 * 3 * 3 + 64 = 18496
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 64 * 64 * 3 * 3 + 64 = 36928

# 분류 (Classification)
model.add(layers.Flatten()) # 576개 벡터로 Flatten
model.add(layers.Dense(64, activation='relu')) # 576 * 64 + 64 = 36928
model.add(layers.Dense(10, activation='softmax')) # 64 * 10 + 10 = 650

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

### 3) 모델 컴파일과 훈련하기

In [3]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1432 - accuracy: 0.9561
Epoch 2/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0469 - accuracy: 0.9858
Epoch 3/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0327 - accuracy: 0.9898
Epoch 4/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0256 - accuracy: 0.9919
Epoch 5/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0203 - accuracy: 0.9934


###4) 모델 평가

In [4]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 1s - loss: 0.0313 - accuracy: 0.9909
0.9908999800682068


---

# 2 CNN for Sentence Classification

##1) 네이버 영화 리뷰 다운로드

In [1]:
# 네이버 영화 리뷰 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

import pandas as pd
import numpy as np
df = pd.read_csv("./ratings.txt",sep='\t').dropna()
df.head(5)

--2020-08-23 09:54:08--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2020-08-23 09:54:08--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt.3’

ratings.txt.3       100%[===================>]  18.61M  63.2MB/s    in 0.3s    

2020-08-23 09:54:09 (63.2 MB/s) - ‘ratings.txt.3’ saved [19515078/19515078]



,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


##2) 전처리

In [2]:
import json
from tensorflow.keras import preprocessing

def preprocess(x, y, padding_size = 128, oov_token="<UNK>", vocab_file = "vocab.json") :  
  preprocessor = preprocessing.text.Tokenizer(oov_token=oov_token) #토큰화
  preprocessor.fit_on_texts(x)
  x = preprocessor.texts_to_sequences(x) #시퀀스로 변환
  vocab = preprocessor.word_index #단어:인덱스 dictionary
  json.dump(vocab, open(vocab_file, 'w'), ensure_ascii=False)
  x = preprocessing.sequence.pad_sequences(x, maxlen=padding_size, padding='post', truncating='post')

  return x, y, vocab

x_train, y_train, vocab = preprocess(df["document"].tolist(), df["label"].tolist())
x_train = x_train[:10000]
y_train = y_train[:10000]

##3) 모델 정의

<img src="https://miro.medium.com/max/770/0*wigQtmJiv0bddwPI." width="500" />

In [5]:
from tensorflow import keras

def CNNforText( num_classes,  #클래스 갯수
          vocab_size,
          embed_size = 512, #단어 임베딩 사이즈                 
          filter_sizes = [3,4,5],
          regularizers_lambda = 0.01, #L2 regulation parameter
          dropout =  0.5,
          feature_size = 128, #문장 시퀀스 길이
          num_filters = 128 #필터 개수 (필터사이즈와 같음). mhlee 하나로 통일하자
) :
          

  inputs = keras.Input(shape=(feature_size,), name='input_data')
  embed_initer = keras.initializers.RandomUniform(minval=-1, maxval=1)
  #sequence 임베딩
  embed = keras.layers.Embedding(vocab_size, embed_size,
                                  embeddings_initializer=embed_initer,
                                  input_length=feature_size,
                                  name='embedding')(inputs)
                                  
  embed = keras.layers.Reshape((feature_size, embed_size, 1), name='add_channel')(embed)

  pool_outputs = []

  #filter 별로 모델 구성
  for filter_size in filter_sizes :
    #convolution
    filter_shape = (filter_size, embed_size)
    conv = keras.layers.Conv2D(num_filters, filter_shape, strides=(1, 1), padding='valid',
                                data_format='channels_last', activation='relu',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                name='convolution_{:d}'.format(filter_size))(embed)
    #max pooling
    max_pool_shape = (feature_size - filter_size + 1, 1)
    pool = keras.layers.MaxPool2D(pool_size=max_pool_shape,
                                  strides=(1, 1), padding='valid',
                                  data_format='channels_last',
                                  name='max_pooling_{:d}'.format(filter_size))(conv)
    pool_outputs.append(pool)

  pool_outputs = keras.layers.concatenate(pool_outputs, axis=-1, name='concatenate')
  pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(pool_outputs)
  pool_outputs = keras.layers.Dropout(dropout, name='dropout')(pool_outputs)

  outputs = keras.layers.Dense(num_classes, activation='softmax',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                kernel_regularizer=keras.regularizers.l2(regularizers_lambda),
                                bias_regularizer=keras.regularizers.l2(regularizers_lambda),
                                name='dense')(pool_outputs)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.summary()
  return model, num_classes

model, num_classes = CNNforText(len(np.unique(y_train)), len(vocab))

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 512)     188943872   input_data[0][0]                 
__________________________________________________________________________________________________
add_channel (Reshape)           (None, 128, 512, 1)  0           embedding[0][0]                  
__________________________________________________________________________________________________
convolution_3 (Conv2D)          (None, 126, 1, 128)  196736      add_channel[0][0]                
_______________________________________________________________________________________

##4) 학습

In [6]:
import os
import time
import tensorflow as tf

def train(model, x_train, y_train, num_classes
          , batch_size = 64, epochs = 1, fraction_validation = 0.05, results_dir = "./result/", save_path = "model") :
  timestamp = time.strftime("%Y-%m-%d-%H-%M", time.localtime(time.time()))
  path = os.path.join(results_dir, timestamp)
  if not os.path.exists(path) :    
    path_log = os.path.join(path, 'log/')
    os.makedirs(path_log)

  model.compile(tf.optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
  #모델 구조 이미지 파일로 저장
  keras.utils.plot_model(model, show_shapes=True, to_file=os.path.join(path, "model.jpg"))
  y_train = tf.one_hot(y_train, num_classes)
  tb_callback = keras.callbacks.TensorBoard(path_log,
                                            histogram_freq=0.1, write_graph=True,
                                            write_images=True,
                                            embeddings_freq=0.5, update_freq='batch')

  history = model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs,
                                callbacks=[tb_callback], validation_split=fraction_validation, shuffle=True)
  
  #모델 저장
  keras.models.save_model(model, save_path)
  print(history.history)

  return model, path_log

model, path_log = train(model, x_train, y_train, num_classes)

  1/149 [..............................] - ETA: 0s - loss: 0.0215 - accuracy: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
149/149 [==============================] - 282s 2s/step - loss: 0.0740 - accuracy: 0.0000e+00 - val_loss: 0.2023 - val_accuracy: 0.0000e+00
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model/assets
{'loss': [0.07399430871009827], 'accuracy': [0.0], 'val_loss': [0.20230326056480408], 'val_accuracy': [0.0]}


In [7]:
%load_ext tensorboard
%tensorboard --logdir {path_log}

##5) 평가

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def test(model, x_test, y_test):
    y_pred_one_hot = model.predict(x=x_test, batch_size=1, verbose=1)
    y_pred = tf.math.argmax(y_pred_one_hot, axis=1)

    print('\nTest accuracy: {}\n'.format(accuracy_score(y_test, y_pred)))
    print('Classification report:')
    target_names = ['class {:d}'.format(i) for i in np.arange(args.num_classes)]
    print(classification_report(y_test, y_pred, target_names=target_names, digits=4))